In [ ]:
! pip install -q mne
! pip install -q mne_icalabel
! pip install -q EDFlib-Python

In [ ]:
!unzip /content/drive/MyDrive/EEG_project/eeg_depression_raw.zip -d eeg_depression ##put your data path

In [ ]:
!mkdir /content/drive/MyDrive/EEG_project/depression/eeg_preprocessed_new

In [ ]:
!mkdir /content/drive/MyDrive/EEG_project/depression/raw_signal
!mkdir /content/drive/MyDrive/EEG_project/depression/cleaned_signal

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import mne
from mne.preprocessing import ICA
from mne_icalabel.iclabel import iclabel_label_components, get_iclabel_features

import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
channels = ['EEG Fp1-LE','EEG F3-LE','EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE',
                   'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Fz-LE', 'EEG Cz-LE', 'EEG Pz-LE']
renamed_chann = [x.split('-')[0].split(' ')[1] for x in channels]

chann_map = {channels[i]: renamed_chann[i] for i in range(len(channels))}

In [ ]:
for f in tqdm(os.listdir('/content/eeg_depression')):
  fp = '/content/eeg_depression/'+f
  num = int(re.findall(r'\d+', f)[-1])
  num_str = str(num//10)+str(num%10)
  if "MDD" in f:
    group="MDD"
  else:
    group="HC"
  if "EC" in f:
    cond = "EC"
  elif "EO" in f:
    cond = "EO"
  else:
    continue
  preprocessed_name = group+'_S'+num_str+'_'+cond
  preprocessed_fname = '/content/drive/MyDrive/EEG_project/depression/eeg_preprocessed_new/'+preprocessed_name+'.edf'

  raw = mne.io.read_raw_edf(fp, preload=True, verbose=False)
  raw.pick_channels(channels, ordered=True)
  raw.rename_channels(chann_map)
  raw.set_montage("standard_1005", match_case=False, on_missing='warn')

  raw.filter(l_freq=0.5, h_freq=70, verbose=False)
  raw.notch_filter(50, verbose=False)
  raw.crop(10, include_tmax=False)

  ica = ICA(n_components=19, max_iter='auto', random_state=97)
  ica.fit(raw)

  # ica.plot_sources(raw);
  # ica.plot_components();

  ic_map = {0:'Brain',1:'Muscle', 2:'Eye', 3:'Heart', 4:'Line Noise', 5:'Channel Noise', 6:'Other'}
  label = np.argmax(iclabel_label_components(raw, ica), axis=1)
  label_class = [ic_map[label[i]] for i in range(len(label))]
  print(label_class)

  ex = np.where(label!=0)[0].tolist()
  print(ex)

  ica.exclude = ex

  reconst_raw = raw.copy()
  ica.apply(reconst_raw)

  fig1 = raw.plot(start=100);
  fig2 = reconst_raw.plot(start=100);

  fig1.savefig('/content/drive/MyDrive/EEG_project/depression/raw_signal/raw_'+preprocessed_name+'.jpg')
  fig2.savefig('/content/drive/MyDrive/EEG_project/depression/cleaned_signal/cleaned_'+preprocessed_name+'.jpg')


  mne.export.export_raw(preprocessed_fname, reconst_raw, overwrite=True, verbose=False)